In [41]:
try:
    get_ipython().run_line_magic("load_ext", "lab_black")
    %config InlineBackendß.figure_format = 'retina'
except:
    print("Could not load lab_black")

# Import packages
import numpy as np
import pandas as pd

pd.set_option("display.max_rows", 500)
pd.options.display.max_colwidth = 2500
pd.set_option("display.max_columns", None)

import sys
import os

from path import Path

from sklearn.preprocessing import LabelEncoder

import importlib

import re

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [42]:
# import modules

module_path = str(Path.joinpath(Path(os.getcwd()).parent.parent, Path("src")))
sys.path.append(os.path.abspath(module_path))

from wce.logger import get_logger
from wce.config import get_config
import wce.preprocess as preprocess
import wce.visualize as visualize

In [43]:
# define logger
logger = get_logger()
logger.info("Logging started")

# load config
cfg = get_config()

2023-04-11 10:23:24,646 - root - [INFO] - [<FileHandler /Users/antonvoskresenskii/Yandex.Disk.localized/ds/aramco/connectivity/src/wce/log.log (INFO)>, <StreamHandler stderr (INFO)>, <FileHandler /Users/antonvoskresenskii/Yandex.Disk.localized/ds/aramco/connectivity/src/wce/log.log (INFO)>, <StreamHandler stderr (INFO)>, <FileHandler /Users/antonvoskresenskii/Yandex.Disk.localized/ds/aramco/connectivity/src/wce/log.log (INFO)>, <StreamHandler stderr (INFO)>, <FileHandler /Users/antonvoskresenskii/Yandex.Disk.localized/ds/aramco/connectivity/src/wce/log.log (INFO)>, <StreamHandler stderr (INFO)>] [MainThread]
2023-04-11 10:23:24,646 - root - [INFO] - [<FileHandler /Users/antonvoskresenskii/Yandex.Disk.localized/ds/aramco/connectivity/src/wce/log.log (INFO)>, <StreamHandler stderr (INFO)>, <FileHandler /Users/antonvoskresenskii/Yandex.Disk.localized/ds/aramco/connectivity/src/wce/log.log (INFO)>, <StreamHandler stderr (INFO)>, <FileHandler /Users/antonvoskresenskii/Yandex.Disk.localized/

In [44]:
path_to_data = Path.joinpath(
    Path(os.getcwd()).parent.parent,
    Path("src") / Path("wce") / cfg["data"]["path_to_data"],
)

path_to_raw_data = Path.joinpath(path_to_data, cfg["data"]["path_to_raw"])

path_to_processed_data = Path.joinpath(path_to_data, cfg["data"]["path_to_processed"])

# Dataset

In [45]:
# load appraisal well data

path = Path.joinpath(
    path_to_raw_data,
    "Volve production data.xlsx",
)
logger.info(f"{path}")

data = pd.read_excel(path, sheet_name="Daily Production Data")

data.dropna(axis=1, how="all", inplace=True)

logger.info(f"shape: {data.shape}")

data.columns = [col.lower() for col in data.columns]

drop_cols = [c for c in list(data) if data[c].nunique() <= 1]
drop_cols = [c for c in drop_cols if c not in [cfg["data"]["group"]]]

data.drop(columns=drop_cols, inplace=True)
logger.info(f"shape: {data.shape}")

data.head()

2023-04-11 10:23:24,702 - root - [INFO] - /Users/antonvoskresenskii/Yandex.Disk.localized/ds/aramco/connectivity/src/wce/data/raw/Volve production data.xlsx [MainThread]
2023-04-11 10:23:24,702 - root - [INFO] - /Users/antonvoskresenskii/Yandex.Disk.localized/ds/aramco/connectivity/src/wce/data/raw/Volve production data.xlsx [MainThread]
2023-04-11 10:23:24,702 - root - [INFO] - /Users/antonvoskresenskii/Yandex.Disk.localized/ds/aramco/connectivity/src/wce/data/raw/Volve production data.xlsx [MainThread]
2023-04-11 10:23:24,702 - root - [INFO] - /Users/antonvoskresenskii/Yandex.Disk.localized/ds/aramco/connectivity/src/wce/data/raw/Volve production data.xlsx [MainThread]
2023-04-11 10:23:26,475 - root - [INFO] - shape: (15832, 24) [MainThread]
2023-04-11 10:23:26,475 - root - [INFO] - shape: (15832, 24) [MainThread]
2023-04-11 10:23:26,475 - root - [INFO] - shape: (15832, 24) [MainThread]
2023-04-11 10:23:26,475 - root - [INFO] - shape: (15832, 24) [MainThread]
2023-04-11 10:23:26,483 

,dateprd,well_bore_code,npd_well_bore_code,npd_well_bore_name,on_stream_hrs,avg_downhole_pressure,avg_downhole_temperature,avg_dp_tubing,avg_annulus_press,avg_choke_size_p,avg_whp_p,avg_wht_p,dp_choke_size,bore_oil_vol,bore_gas_vol,bore_wat_vol,bore_wi_vol,flow_kind,well_type
0,2014-04-07,NO 15/9-F-1 C,7405,15/9-F-1 C,0.0,0.00000,0.00000,0.00000,0.0,0.000000,0.00000,0.00000,0.00000,0.0,0.0,0.0,NaN,production,OP
1,2014-04-08,NO 15/9-F-1 C,7405,15/9-F-1 C,0.0,NaN,NaN,NaN,0.0,1.003059,0.00000,0.00000,0.00000,0.0,0.0,0.0,NaN,production,OP
2,2014-04-09,NO 15/9-F-1 C,7405,15/9-F-1 C,0.0,NaN,NaN,NaN,0.0,0.979008,0.00000,0.00000,0.00000,0.0,0.0,0.0,NaN,production,OP
3,2014-04-10,NO 15/9-F-1 C,7405,15/9-F-1 C,0.0,NaN,NaN,NaN,0.0,0.545759,0.00000,0.00000,0.00000,0.0,0.0,0.0,NaN,production,OP
4,2014-04-11,NO 15/9-F-1 C,7405,15/9-F-1 C,0.0,310.37614,96.87589,277.27826,0.0,1.215987,33.09788,10.47992,33.07195,0.0,0.0,0.0,NaN,production,OP


In [46]:
data["npd_well_bore_name"].unique()

array(['15/9-F-1 C', '15/9-F-11', '15/9-F-12', '15/9-F-14', '15/9-F-15 D',
       '15/9-F-4', '15/9-F-5', '15/9-F-15'], dtype=object)

In [47]:
logger.info(f"shape: {data.shape}")
data = data[
    ~((data["well_type"] == "WI") & (data["npd_well_bore_name"] == "15/9-F-1 C"))
]
data = data[~((data["well_type"] == "OP") & (data["npd_well_bore_name"] == "15/9-F-5"))]
data = data[~(data["npd_well_bore_name"] == "15/9-F-15")]

logger.info(f"shape: {data.shape}")

2023-04-11 10:23:26,527 - root - [INFO] - shape: (15832, 19) [MainThread]
2023-04-11 10:23:26,527 - root - [INFO] - shape: (15832, 19) [MainThread]
2023-04-11 10:23:26,527 - root - [INFO] - shape: (15832, 19) [MainThread]
2023-04-11 10:23:26,527 - root - [INFO] - shape: (15832, 19) [MainThread]
2023-04-11 10:23:26,537 - root - [INFO] - shape: (15489, 19) [MainThread]
2023-04-11 10:23:26,537 - root - [INFO] - shape: (15489, 19) [MainThread]
2023-04-11 10:23:26,537 - root - [INFO] - shape: (15489, 19) [MainThread]
2023-04-11 10:23:26,537 - root - [INFO] - shape: (15489, 19) [MainThread]


In [48]:
data[cfg["data"]["cat"]] = data.apply(
    lambda x: re.split("-", x["npd_well_bore_name"])[1]
    + "-"
    + re.split("-", x["npd_well_bore_name"])[2],
    axis=1,
)

dictionary = dict(zip(data["npd_well_bore_name"], data[cfg["data"]["cat"]]))
print(dictionary)

data.drop(
    columns=["well_bore_code", "npd_well_bore_code", "npd_well_bore_name", "flow_kind"],
    inplace=True,
)
logger.info(f"shape: {data.shape}")

data.rename(
    columns={"well_type": cfg["data"]["group"], "dateprd": cfg["data"]["date"]},
    inplace=True,
)

logger.info(f"shape: {data.shape}")

data.reset_index(drop=True, inplace=True)

logger.info(f"shape: {data.shape}")
data.head()

2023-04-11 10:23:26,629 - root - [INFO] - shape: (15489, 16) [MainThread]
2023-04-11 10:23:26,629 - root - [INFO] - shape: (15489, 16) [MainThread]
2023-04-11 10:23:26,629 - root - [INFO] - shape: (15489, 16) [MainThread]
2023-04-11 10:23:26,629 - root - [INFO] - shape: (15489, 16) [MainThread]
2023-04-11 10:23:26,630 - root - [INFO] - shape: (15489, 16) [MainThread]
2023-04-11 10:23:26,630 - root - [INFO] - shape: (15489, 16) [MainThread]
2023-04-11 10:23:26,630 - root - [INFO] - shape: (15489, 16) [MainThread]
2023-04-11 10:23:26,630 - root - [INFO] - shape: (15489, 16) [MainThread]
2023-04-11 10:23:26,632 - root - [INFO] - shape: (15489, 16) [MainThread]
2023-04-11 10:23:26,632 - root - [INFO] - shape: (15489, 16) [MainThread]
2023-04-11 10:23:26,632 - root - [INFO] - shape: (15489, 16) [MainThread]
2023-04-11 10:23:26,632 - root - [INFO] - shape: (15489, 16) [MainThread]


{'15/9-F-1 C': 'F-1 C', '15/9-F-11': 'F-11', '15/9-F-12': 'F-12', '15/9-F-14': 'F-14', '15/9-F-15 D': 'F-15 D', '15/9-F-4': 'F-4', '15/9-F-5': 'F-5'}


,date,on_stream_hrs,avg_downhole_pressure,avg_downhole_temperature,avg_dp_tubing,avg_annulus_press,avg_choke_size_p,avg_whp_p,avg_wht_p,dp_choke_size,bore_oil_vol,bore_gas_vol,bore_wat_vol,bore_wi_vol,group,cat
0,2014-04-07,0.0,0.00000,0.00000,0.00000,0.0,0.000000,0.00000,0.00000,0.00000,0.0,0.0,0.0,NaN,OP,F-1 C
1,2014-04-08,0.0,NaN,NaN,NaN,0.0,1.003059,0.00000,0.00000,0.00000,0.0,0.0,0.0,NaN,OP,F-1 C
2,2014-04-09,0.0,NaN,NaN,NaN,0.0,0.979008,0.00000,0.00000,0.00000,0.0,0.0,0.0,NaN,OP,F-1 C
3,2014-04-10,0.0,NaN,NaN,NaN,0.0,0.545759,0.00000,0.00000,0.00000,0.0,0.0,0.0,NaN,OP,F-1 C
4,2014-04-11,0.0,310.37614,96.87589,277.27826,0.0,1.215987,33.09788,10.47992,33.07195,0.0,0.0,0.0,NaN,OP,F-1 C


In [49]:
d = {
    "15/9-F-4": (435654.01, 6477677.22),
    "15/9-F-5": (436403.89, 6478703.6),
    "15/9-F-14": (436028.94, 6478876.54),
    "15/9-F-1 C": (435584.78, 6479497.27),
    "15/9-F-11": (436551.57, 6478399.73),
    "15/9-F-12": (435335.51, 6478201.54),
    "15/9-F-15 D": (436650.23, 6478716.35),
    # "15/9-F-15": (434105.85, 6476758.93),
}
df = pd.DataFrame.from_dict(d).T
df.reset_index(inplace=True)
df.columns = ["NPD_WELL_BORE_NAME", "x", "y"]

logger.info(f"shape: {df.shape}")

df["cat"] = df["NPD_WELL_BORE_NAME"].map(dictionary)

df.drop(columns=["NPD_WELL_BORE_NAME"], inplace=True)

path = Path.joinpath(
    path_to_processed_data,
    "coords_volve.csv",
)
logger.info(f"{path}")
df.to_csv(path, index=False)

logger.info(f"shape: {df.shape}")
df

2023-04-11 10:23:26,674 - root - [INFO] - shape: (7, 3) [MainThread]
2023-04-11 10:23:26,674 - root - [INFO] - shape: (7, 3) [MainThread]
2023-04-11 10:23:26,674 - root - [INFO] - shape: (7, 3) [MainThread]
2023-04-11 10:23:26,674 - root - [INFO] - shape: (7, 3) [MainThread]
2023-04-11 10:23:26,677 - root - [INFO] - /Users/antonvoskresenskii/Yandex.Disk.localized/ds/aramco/connectivity/src/wce/data/processed/coords_volve.csv [MainThread]
2023-04-11 10:23:26,677 - root - [INFO] - /Users/antonvoskresenskii/Yandex.Disk.localized/ds/aramco/connectivity/src/wce/data/processed/coords_volve.csv [MainThread]
2023-04-11 10:23:26,677 - root - [INFO] - /Users/antonvoskresenskii/Yandex.Disk.localized/ds/aramco/connectivity/src/wce/data/processed/coords_volve.csv [MainThread]
2023-04-11 10:23:26,677 - root - [INFO] - /Users/antonvoskresenskii/Yandex.Disk.localized/ds/aramco/connectivity/src/wce/data/processed/coords_volve.csv [MainThread]
2023-04-11 10:23:26,679 - root - [INFO] - shape: (7, 3) [Mai

,x,y,cat
0,435654.01,6477677.22,F-4
1,436403.89,6478703.60,F-5
2,436028.94,6478876.54,F-14
3,435584.78,6479497.27,F-1 C
4,436551.57,6478399.73,F-11
5,435335.51,6478201.54,F-12
6,436650.23,6478716.35,F-15 D


In [50]:
importlib.reload(preprocess)

data = preprocess.shorten_column_names(data)
data.head()

2023-04-11 10:23:26,715 - wce.preprocess - [INFO] - Change column names. [MainThread]
2023-04-11 10:23:26,715 - wce.preprocess - [INFO] - Change column names. [MainThread]
2023-04-11 10:23:26,715 - wce.preprocess - [INFO] - Change column names. [MainThread]
2023-04-11 10:23:26,715 - wce.preprocess - [INFO] - Change column names. [MainThread]


,date,onStream,bhp,bht,avg_dp_tubing,avg_annulus_press,choke,avg_whp_p,avg_wht_p,dp_choke_size,oil,bore_gas_vol,water,waterInj,group,cat
0,2014-04-07,0.0,0.00000,0.00000,0.00000,0.0,0.000000,0.00000,0.00000,0.00000,0.0,0.0,0.0,NaN,OP,F-1 C
1,2014-04-08,0.0,NaN,NaN,NaN,0.0,1.003059,0.00000,0.00000,0.00000,0.0,0.0,0.0,NaN,OP,F-1 C
2,2014-04-09,0.0,NaN,NaN,NaN,0.0,0.979008,0.00000,0.00000,0.00000,0.0,0.0,0.0,NaN,OP,F-1 C
3,2014-04-10,0.0,NaN,NaN,NaN,0.0,0.545759,0.00000,0.00000,0.00000,0.0,0.0,0.0,NaN,OP,F-1 C
4,2014-04-11,0.0,310.37614,96.87589,277.27826,0.0,1.215987,33.09788,10.47992,33.07195,0.0,0.0,0.0,NaN,OP,F-1 C


In [51]:
for col in ["choke", "oil", "water"]:
    data[col] = np.where(data["cat"] == "F-5", np.nan, data[col])

data["water"] = np.where(data["water"] < 0, np.nan, data["water"])

In [52]:
data["liquid"] = data["oil"] + data["water"]
data["watercut"] = data["water"] / data["liquid"]

In [53]:
data.groupby([cfg["data"]["cat"], cfg["data"]["group"]]).agg(
    {
        "waterInj": "count",
        "bhp": "count",
        "choke": "count",
        "oil": "count",
        "water": "count",
        "liquid": "count",
        "watercut": "count",
    }
)

,,waterInj,bhp,choke,oil,water,liquid,watercut
cat,group,,,,,,,
F-1 C,OP,0,742,745,745,745,745,429
F-11,OP,0,1159,1163,1165,1165,1165,1123
F-12,OP,0,3050,3012,3056,3054,3054,2835
F-14,OP,0,3050,2860,3056,3054,3054,2722
F-15 D,OP,0,978,978,978,978,978,767
F-4,WI,2990,0,0,0,0,0,0
F-5,WI,2716,0,0,0,0,0,0


In [54]:
d = {
    "OP": "P",
    "WI": "I",
}
data["group"] = data["group"].map(d)

In [55]:
path = Path.joinpath(
    path_to_processed_data,
    "data_volve.csv",
)
logger.info(f"{path}")
data.to_csv(path, index=False)

2023-04-11 10:23:26,841 - root - [INFO] - /Users/antonvoskresenskii/Yandex.Disk.localized/ds/aramco/connectivity/src/wce/data/processed/data_volve.csv [MainThread]
2023-04-11 10:23:26,841 - root - [INFO] - /Users/antonvoskresenskii/Yandex.Disk.localized/ds/aramco/connectivity/src/wce/data/processed/data_volve.csv [MainThread]
2023-04-11 10:23:26,841 - root - [INFO] - /Users/antonvoskresenskii/Yandex.Disk.localized/ds/aramco/connectivity/src/wce/data/processed/data_volve.csv [MainThread]
2023-04-11 10:23:26,841 - root - [INFO] - /Users/antonvoskresenskii/Yandex.Disk.localized/ds/aramco/connectivity/src/wce/data/processed/data_volve.csv [MainThread]
